<a href="https://colab.research.google.com/github/julioreis-dev/data_science/blob/main/ApacheSparkAula01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
# instalar as dependências
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# instalar pyspark
pip install -q pyspark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [46.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,419 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadc

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
%%bash
# Dowload dos dados utilizados
mkdir titanic
curl https://raw.githubusercontent.com/neylsoncrepalde/titanic_data_with_semicolon/main/titanic.csv -o titanic/titanic.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60326  100 60326    0     0   199k      0 --:--:-- --:--:-- --:--:--  199k


In [4]:
# Importar os módulos necessários
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# Iniciar a sessão Spark
spark = SparkSession.builder.getOrCreate()

In [5]:
# Leitura de dados
# titanic = (
#     spark
#     .read
#     .format("csv")
#     .option("delimiter", ";")
#     .option("header", True)
#     .option("inferSchema", True)
#     .load("titanic")
# )
# titanic.printSchema()

# Modo alternativo
titanic = (
    spark
    .read
    .csv("titanic", header=True, sep=";", inferSchema=True)
)
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
titanic.show(truncate=False)

+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22.0|1    |0    |A/5 21171       |7.25   |NULL |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38.0|1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26.0|0    |0    |STON/O2. 3101282|7.925  |NULL |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)           |female|35.0|1  

# Escrever os dados

In [7]:
(
    titanic
    .write
    .mode("overwrite")
    .save("titanic_parquet")
)

In [8]:
# Selecionar colunas
titanic.select("Name", "Sex", "Pclass", "Survived").show(n=10)

+--------------------+------+------+--------+
|                Name|   Sex|Pclass|Survived|
+--------------------+------+------+--------+
|Braund, Mr. Owen ...|  male|     3|       0|
|Cumings, Mrs. Joh...|female|     1|       1|
|Heikkinen, Miss. ...|female|     3|       1|
|Futrelle, Mrs. Ja...|female|     1|       1|
|Allen, Mr. Willia...|  male|     3|       0|
|    Moran, Mr. James|  male|     3|       0|
|McCarthy, Mr. Tim...|  male|     1|       0|
|Palsson, Master. ...|  male|     3|       0|
|Johnson, Mrs. Osc...|female|     3|       1|
|Nasser, Mrs. Nich...|female|     2|       1|
+--------------------+------+------+--------+
only showing top 10 rows



In [9]:
titanic.where("Sex = 'male' AND Survived = 1").show()

+-----------+--------+------+--------------------+----+----+-----+-----+--------+-------+-------+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|  Ticket|   Fare|  Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+--------+-------+-------+--------+
|         18|       1|     2|Williams, Mr. Cha...|male|NULL|    0|    0|  244373|   13.0|   NULL|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|male|34.0|    0|    0|  248698|   13.0|    D56|       S|
|         24|       1|     1|Sloper, Mr. Willi...|male|28.0|    0|    0|  113788|   35.5|     A6|       S|
|         37|       1|     3|    Mamee, Mr. Hanna|male|NULL|    0|    0|    2677| 7.2292|   NULL|       C|
|         56|       1|     1|   Woolner, Mr. Hugh|male|NULL|    0|    0|   19947|   35.5|    C52|       S|
|         66|       1|     3|Moubarek, Master....|male|NULL|    1|    1|    2661|15.2458|   NULL|       C|
|         75|       1|     3|       B

In [10]:
titanic.agg(
    f.mean("Age").alias('med_idade'),
    f.min('Age').alias('idade_minima'),
    f.max('Age').alias('idade_maxima'),
    f.stddev('Age').alias('desvio_padrao_idade')
).show()

+-----------------+------------+------------+-------------------+
|        med_idade|idade_minima|idade_maxima|desvio_padrao_idade|
+-----------------+------------+------------+-------------------+
|29.69911764705882|        0.42|        80.0| 14.526497332334035|
+-----------------+------------+------------+-------------------+



In [11]:
# Métricas agrupadas
(
    titanic
    .groupBy('Sex')
    .agg(
        f.mean("Age").alias('med_idade'),
        f.min('Age').alias('idade_minima'),
        f.max('Age').alias('idade_maxima'),
        f.stddev('Age').alias('desvio_padrao_idade')
    ).show()
)

+------+------------------+------------+------------+-------------------+
|   Sex|         med_idade|idade_minima|idade_maxima|desvio_padrao_idade|
+------+------------------+------------+------------+-------------------+
|female|27.915708812260537|        0.75|        63.0| 14.110146457544133|
|  male| 30.72664459161148|        0.42|        80.0| 14.678200823816608|
+------+------------------+------------+------------+-------------------+



In [12]:
# Modo alternativo
titanic.groupBy('Sex').agg(
        f.mean("Age").alias('med_idade'),
        f.min('Age').alias('idade_minima'),
        f.max('Age').alias('idade_maxima'),
        f.stddev('Age').alias('desvio_padrao_idade')
).show()

+------+------------------+------------+------------+-------------------+
|   Sex|         med_idade|idade_minima|idade_maxima|desvio_padrao_idade|
+------+------------------+------------+------------+-------------------+
|female|27.915708812260537|        0.75|        63.0| 14.110146457544133|
|  male| 30.72664459161148|        0.42|        80.0| 14.678200823816608|
+------+------------------+------------+------------+-------------------+



In [13]:
# Ordenar os resultados
(
    titanic
    .groupBy('Sex', 'Pclass', 'Survived')
    .agg(
        f.mean("Age").alias('med_idade'),
        f.min('Age').alias('idade_minima'),
        f.max('Age').alias('idade_maxima'),
        f.stddev('Age').alias('desvio_padrao_idade')
    )
    .orderBy('Sex', 'Pclass', 'Survived')
    .show()
)

+------+------+--------+------------------+------------+------------+-------------------+
|   Sex|Pclass|Survived|         med_idade|idade_minima|idade_maxima|desvio_padrao_idade|
+------+------+--------+------------------+------------+------------+-------------------+
|female|     1|       0|25.666666666666668|         2.0|        50.0|  24.00694344004112|
|female|     1|       1|  34.9390243902439|        14.0|        63.0| 13.223013572001038|
|female|     2|       0|              36.0|        24.0|        57.0| 12.915107432770352|
|female|     2|       1|28.080882352941178|         2.0|        55.0|  12.76469311243358|
|female|     3|       0|23.818181818181817|         2.0|        48.0| 12.833464514612233|
|female|     3|       1|19.329787234042552|        0.75|        63.0| 12.303245681590393|
|  male|     1|       0| 44.58196721311475|        18.0|        71.0| 14.457748881905525|
|  male|     1|       1|36.248000000000005|        0.92|        80.0| 14.936743749765334|
|  male|  

In [14]:
# Registrar o dataset como uma view para usar como SQL
titanic.createOrReplaceTempView('titanic_sql')

In [15]:
spark.sql("""
  SELECT  Name, Sex, Pclass, Age
  FROM titanic_sql
""").show()

+--------------------+------+------+----+
|                Name|   Sex|Pclass| Age|
+--------------------+------+------+----+
|Braund, Mr. Owen ...|  male|     3|22.0|
|Cumings, Mrs. Joh...|female|     1|38.0|
|Heikkinen, Miss. ...|female|     3|26.0|
|Futrelle, Mrs. Ja...|female|     1|35.0|
|Allen, Mr. Willia...|  male|     3|35.0|
|    Moran, Mr. James|  male|     3|NULL|
|McCarthy, Mr. Tim...|  male|     1|54.0|
|Palsson, Master. ...|  male|     3| 2.0|
|Johnson, Mrs. Osc...|female|     3|27.0|
|Nasser, Mrs. Nich...|female|     2|14.0|
|Sandstrom, Miss. ...|female|     3| 4.0|
|Bonnell, Miss. El...|female|     1|58.0|
|Saundercock, Mr. ...|  male|     3|20.0|
|Andersson, Mr. An...|  male|     3|39.0|
|Vestrom, Miss. Hu...|female|     3|14.0|
|Hewlett, Mrs. (Ma...|female|     2|55.0|
|Rice, Master. Eugene|  male|     3| 2.0|
|Williams, Mr. Cha...|  male|     2|NULL|
|Vander Planke, Mr...|female|     3|31.0|
|Masselmani, Mrs. ...|female|     3|NULL|
+--------------------+------+-----

In [16]:
spark.sql("""
  SELECT  Name, Sex, Pclass, Age
  FROM titanic_sql
  WHERE Sex = 'male'
""").show()

+--------------------+----+------+----+
|                Name| Sex|Pclass| Age|
+--------------------+----+------+----+
|Braund, Mr. Owen ...|male|     3|22.0|
|Allen, Mr. Willia...|male|     3|35.0|
|    Moran, Mr. James|male|     3|NULL|
|McCarthy, Mr. Tim...|male|     1|54.0|
|Palsson, Master. ...|male|     3| 2.0|
|Saundercock, Mr. ...|male|     3|20.0|
|Andersson, Mr. An...|male|     3|39.0|
|Rice, Master. Eugene|male|     3| 2.0|
|Williams, Mr. Cha...|male|     2|NULL|
|Fynney, Mr. Joseph J|male|     2|35.0|
|Beesley, Mr. Lawr...|male|     2|34.0|
|Sloper, Mr. Willi...|male|     1|28.0|
|Emir, Mr. Farred ...|male|     3|NULL|
|Fortune, Mr. Char...|male|     1|19.0|
| Todoroff, Mr. Lalio|male|     3|NULL|
|Uruchurtu, Don. M...|male|     1|40.0|
|Wheadon, Mr. Edwa...|male|     2|66.0|
|Meyer, Mr. Edgar ...|male|     1|28.0|
|Holverson, Mr. Al...|male|     1|42.0|
|    Mamee, Mr. Hanna|male|     3|NULL|
+--------------------+----+------+----+
only showing top 20 rows



In [17]:
spark.sql("""
  SELECT
    Name, Sex, Pclass, Age
  FROM titanic_sql
  WHERE
    Sex = 'male' and Pclass = 1
""").show(truncate = False)

+-------------------------------+----+------+----+
|Name                           |Sex |Pclass|Age |
+-------------------------------+----+------+----+
|McCarthy, Mr. Timothy J        |male|1     |54.0|
|Sloper, Mr. William Thompson   |male|1     |28.0|
|Fortune, Mr. Charles Alexander |male|1     |19.0|
|Uruchurtu, Don. Manuel E       |male|1     |40.0|
|Meyer, Mr. Edgar Joseph        |male|1     |28.0|
|Holverson, Mr. Alexander Oskar |male|1     |42.0|
|Ostby, Mr. Engelhart Cornelius |male|1     |65.0|
|Woolner, Mr. Hugh              |male|1     |NULL|
|Harris, Mr. Henry Birkhardt    |male|1     |45.0|
|Stewart, Mr. Albert A          |male|1     |NULL|
|Carrau, Mr. Francisco M        |male|1     |28.0|
|Chaffee, Mr. Herbert Fuller    |male|1     |46.0|
|Goldschmidt, Mr. George B      |male|1     |71.0|
|Greenfield, Mr. William Bertram|male|1     |23.0|
|White, Mr. Richard Frasar      |male|1     |21.0|
|Porter, Mr. Walter Chamberlain |male|1     |47.0|
|Baxter, Mr. Quigg Edmond      

In [18]:
spark.sql("""
  SELECT
    AVG(Age) as idade_media,
    MIN(Age) as idade_minima,
    Max(Age) as idade_maxima,
    STDDEV(Age) as desvio_padrao
  FROM titanic_sql
""").show()

+-----------------+------------+------------+------------------+
|      idade_media|idade_minima|idade_maxima|     desvio_padrao|
+-----------------+------------+------------+------------------+
|29.69911764705882|        0.42|        80.0|14.526497332334035|
+-----------------+------------+------------+------------------+



In [19]:
spark.sql("""
  SELECT
    Sex, Pclass, Survived,
    AVG(Age) as idade_media,
     Max(Age) as idade_maxima,
    STDDEV(Age) as desvio_padrao
  FROM titanic_sql
  GROUP BY Sex, Pclass, Survived
  ORDER BY Sex, Pclass, Survived
""").show(truncate = False)

+------+------+--------+------------------+------------+------------------+
|Sex   |Pclass|Survived|idade_media       |idade_maxima|desvio_padrao     |
+------+------+--------+------------------+------------+------------------+
|female|1     |0       |25.666666666666668|50.0        |24.00694344004112 |
|female|1     |1       |34.9390243902439  |63.0        |13.223013572001038|
|female|2     |0       |36.0              |57.0        |12.915107432770352|
|female|2     |1       |28.080882352941178|55.0        |12.76469311243358 |
|female|3     |0       |23.818181818181817|48.0        |12.833464514612233|
|female|3     |1       |19.329787234042552|63.0        |12.303245681590393|
|male  |1     |0       |44.58196721311475 |71.0        |14.457748881905525|
|male  |1     |1       |36.248000000000005|80.0        |14.936743749765334|
|male  |2     |0       |33.36904761904762 |70.0        |12.15812461947834 |
|male  |2     |1       |16.022            |62.0        |19.547121527222362|
|male  |3   

## Perguntas
1. Quantos passageiros sobreviveram ao naufrágio do titanic?
2. Quantos sobreviveram entre homens e mulheres?
3. Quantos sobreviveram entre pessoas que voiajavam em classes diferentes?
4. Quantas pessoas sobreviveram ao naufrágio por sexo, classe de viagem e que tinha idade maior do que a média de todo o navio?

#Resposta da pergunta 1

In [20]:
titanic.where('Survived = 1').count()

342

In [21]:
spark.sql("""
  SELECT
    Survived, COUNT(*) as total
  FROM titanic_sql
  WHERE Survived = 1
  GROUP BY Survived
""").show()

+--------+-----+
|Survived|total|
+--------+-----+
|       1|  342|
+--------+-----+



#Resposta da pergunta 2

In [22]:
(
    titanic
    .where('Survived = 1')
    .groupBy('Sex')
    .count()
    .show()
)

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



In [23]:
spark.sql("""
  SELECT
    Sex, Survived, COUNT(*) as total
  FROM titanic_sql
  WHERE Survived = 1
  GROUP BY Sex, Survived
""").show()

+------+--------+-----+
|   Sex|Survived|total|
+------+--------+-----+
|female|       1|  233|
|  male|       1|  109|
+------+--------+-----+



#Resposta da pergunta 3

In [24]:
(
    titanic
    .where('Survived = 1')
    .groupBy('Pclass')
    .count()
    .orderBy('Pclass')
    .show()
)

+------+-----+
|Pclass|count|
+------+-----+
|     1|  136|
|     2|   87|
|     3|  119|
+------+-----+



In [25]:
spark.sql("""
  SELECT
    Pclass, Survived, COUNT(*) as total
  FROM titanic_sql
  WHERE Survived = 1
  GROUP BY Pclass, Survived
  ORDER By Pclass
""").show()

+------+--------+-----+
|Pclass|Survived|total|
+------+--------+-----+
|     1|       1|  136|
|     2|       1|   87|
|     3|       1|  119|
+------+--------+-----+



#Resposta da pergunta 4

In [26]:
(
    titanic
    .where('Survived = 1 and Age > 29.69')
    .groupBy('Sex', 'Pclass')
    .count()
    .orderBy('Sex', 'Pclass')
    .show()
)

+------+------+-----+
|   Sex|Pclass|count|
+------+------+-----+
|female|     1|   54|
|female|     2|   30|
|female|     3|    8|
|  male|     1|   27|
|  male|     2|    5|
|  male|     3|   10|
+------+------+-----+



In [30]:
spark.sql("""
  SELECT
    Sex, Pclass, COUNT(*) as total
  FROM titanic_sql
  WHERE Survived = 1 AND Age > (SELECT AVG(Age) FROM titanic_sql)
  GROUP BY Sex, Pclass
  ORDER By Sex, Pclass
""").show()

+------+------+-----+
|   Sex|Pclass|total|
+------+------+-----+
|female|     1|   54|
|female|     2|   30|
|female|     3|    8|
|  male|     1|   27|
|  male|     2|    5|
|  male|     3|   10|
+------+------+-----+

